# Homework 3 - MongoDB

In [1]:
import json
import os
import pymongo
import pandas as pd
from datetime import date, datetime, timedelta

## Lettura dei dati

In [2]:
dataset_dir = './dataset/'
dati_covid_path = os.path.join(dataset_dir, 'dpc-covid19-ita-regioni.json')
dati_vaccini_path = os.path.join(dataset_dir, 'somministrazioni-vaccini-latest.json')

dati_covid = []
with open(dati_covid_path, 'r') as j:
    dati_covid = json.load(j)

dati_vaccini = []
with open(dati_vaccini_path, 'r') as j:
    dati_vaccini = json.load(j)
dati_vaccini = dati_vaccini['data']

## Creazione del db
Si crea il database e due collection contenenti rispettivamente dati relativi al Covid-19 e alle vaccinazioni.

In [3]:
client = pymongo.MongoClient('localhost', 27017)

db = client['homework3']

collection_covid = db.dati_covid
collection_vaccini = db.dati_vaccini

print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'homework3', 'local']
Available collections:  ['dati_covid', 'dati_vaccini']


Si inseriscono i dati nelle rispettive collection (solo se il database è stato appena creato).

In [4]:
# collection_covid.insert_many(dati_covid)
# collection_vaccini.insert_many(dati_vaccini)

In [5]:
print('Available database: ', client.list_database_names())
print('Available collections: ', db.list_collection_names())

Available database:  ['admin', 'config', 'homework3', 'local']
Available collections:  ['dati_covid', 'dati_vaccini']


## Preprocessing
Si convertono i campi relativi alle date dal formato `string` al formato `date`.

In [6]:
collection_covid.aggregate([
    { "$addFields": {
        "data": {
            "$toDate": "$data"
            }
        } 
    },
    { "$unset": ["casi_da_screening", ]},
    { "$out" : "dati_covid" }
])

In [7]:
collection_vaccini.aggregate([
    { "$addFields": {
        "data_somministrazione": {
            "$toDate": "$data_somministrazione"
            }
        } 
    },
    { "$out" : "dati_vaccini" }
])

## Query

### Attuali positivi e terapie intensive per regione

In [18]:
days_ago = 10
date = datetime.today() - timedelta(days=days_ago)

start = date.replace(hour=0, minute=0, second=0)
end = date.replace(hour=23, minute=59, second=59)

filter = {
    'data': {
        '$gt': start,
        '$lt': end
    }
}

project = {
    'data': 1, 
    'denominazione_regione': 1, 
    'totale_positivi': 1,
    'terapia_intensiva': 1,
    '_id': 0
}

sort = list({
    'totale_positivi': -1
}.items())

result = collection_covid.find(
    filter=filter,
    projection=project,
    sort=sort
)

result_list = list(result)
# len(result_list)

In [19]:
df = pd.DataFrame(result_list)
df

,data,denominazione_regione,terapia_intensiva,totale_positivi
0,2021-05-10 17:00:00,Campania,113,84584
1,2021-05-10 17:00:00,Puglia,183,43461
2,2021-05-10 17:00:00,Lombardia,479,41701
3,2021-05-10 17:00:00,Lazio,270,36868
4,2021-05-10 17:00:00,Emilia-Romagna,187,31379
5,2021-05-10 17:00:00,Sicilia,131,22230
6,2021-05-10 17:00:00,Veneto,126,19036
7,2021-05-10 17:00:00,Toscana,205,17148
8,2021-05-10 17:00:00,Sardegna,41,15475
9,2021-05-10 17:00:00,Calabria,31,13529


### Nuovi positivi nell'ultima settimana per regione

In [21]:
weeks_ago = 1
date = datetime.today() - timedelta(weeks=1)

result = collection_covid.aggregate([
    { "$match": { "data": { "$gte": date } } },

    { "$group": {
        "_id": "$denominazione_regione",
        "nuovi_casi": { "$sum": "$nuovi_positivi" },
        "tamponi" : { "$sum": "$tamponi"}
    }},
    
    { "$sort" : { "nuovi_casi": -1 } }
])

result_list = list(result)
# len(result_list)

In [22]:
df = pd.DataFrame(result_list)
df

,_id,nuovi_casi,tamponi
0,Lombardia,2556,20079301
1,Campania,2228,9255794
2,Lazio,1360,13094671
3,Piemonte,1301,8430226
4,Toscana,1180,8835623
5,Sicilia,1176,8401287
6,Emilia-Romagna,1169,11820011
7,Puglia,1145,4717961
8,Veneto,921,15092514
9,Calabria,580,1637822


### ?

In [98]:
result = collection_covid.aggregate([
    { "$group" : {
        "_id": {
            "anno" : {"$year": "$data"},
            "mese" : {"$month": "$data"},
#             "settimana" : {"$week": "$data"},
            "regione" : "$denominazione_regione"
        },
        "deceduti": { "$max": "$deceduti" },
        "count": { "$sum": 1 }
    }},
    
    { "$project": {
        "anno": "$_id.anno",
        "mese": "$_id.mese",
        "regione": "$_id.regione",
        "deceduti": 1,
        "_id": 0
    }},
    
    { "$group" : {
        "_id": {
            "anno" : "$anno",
            "mese" : "$mese",
        },
        "deceduti": { "$sum": "$deceduti" },
        "count": { "$sum": 1 }
    }},
    
    { "$sort" : {"_id.anno": -1, "_id.mese": -1} }
 ])

result_list = list(result)
len(result_list)

16

In [99]:
df = pd.DataFrame(result_list)
df

,_id,count,deceduti
0,"{'mese': 5, 'anno': 2021}",21,123927
1,"{'mese': 4, 'anno': 2021}",21,120807
2,"{'mese': 3, 'anno': 2021}",21,109346
3,"{'mese': 2, 'anno': 2021}",21,97699
4,"{'mese': 1, 'anno': 2021}",21,88516
5,"{'mese': 12, 'anno': 2020}",21,74159
6,"{'mese': 11, 'anno': 2020}",21,55576
7,"{'mese': 10, 'anno': 2020}",21,38618
8,"{'mese': 9, 'anno': 2020}",21,35894
9,"{'mese': 8, 'anno': 2020}",21,35483
